In [1]:
import os
opj = os.path.join

import matplotlib.pyplot as plt
import pandas as pd
from pytorch_tabnet.tab_model import TabNetRegressor 
import torch

In [2]:
ckpt_dir = "ckpts/no_neutral_8hours"
model_name = "bert-high"
ckpts = []
for file in os.listdir(ckpt_dir):
    if model_name in file:
        fdir = opj(ckpt_dir, file)
        ckpts.append(fdir)
        print(fdir)

ckpts/no_neutral_8hours/bert-high-9.zip
ckpts/no_neutral_8hours/bert-high-1.zip
ckpts/no_neutral_8hours/bert-high-0.zip
ckpts/no_neutral_8hours/bert-high-8.zip
ckpts/no_neutral_8hours/bert-high-7.zip
ckpts/no_neutral_8hours/bert-high-2.zip
ckpts/no_neutral_8hours/bert-high-4.zip
ckpts/no_neutral_8hours/bert-high-6.zip
ckpts/no_neutral_8hours/bert-high-5.zip
ckpts/no_neutral_8hours/bert-high-3.zip


In [3]:
df_dir = "data/8hours/bert.csv"

In [4]:
def getParams(df_dir, seed):
    # load data
    data_dir = df_dir
    chart_df = pd.read_csv(df_dir)
    i1 = 0
    i2 = len(chart_df)-1
    st = "2018-02-14 14:00:00"
    en = "2022-04-15 23:00:00"
    filt = []
    for index, row in chart_df.iterrows():
        if chart_df.iloc[i1]['datetime'] < row.datetime and row.datetime < st:
            i1 = index
        if en < row.datetime and row.datetime < chart_df.iloc[i2]['datetime']:
            i2 = index
    chart_df = chart_df[i1+1:i2]
    chart_df.index = range(len(chart_df))
    
    # Train High Model
    ## train/validation/test split 
    train_size = int(chart_df.shape[0] * 0.8) 
    train_df = chart_df.iloc[:train_size,:] 

    val_size = int(chart_df.shape[0] * 0.1) 
    val_df = chart_df.iloc[train_size:train_size+val_size,:]  

    test_df = chart_df.iloc[train_size+val_size:, :] 
    
    categorical_columns = ["months", "days", "hours"]
    features = train_df.columns
    
    
    cat_idxs = [0, 1, 2] 
    cat_dims = [13, 32, 25] 
    
    tabnet_params = {"cat_idxs":cat_idxs, 
                     "cat_dims":cat_dims, 
                     "cat_emb_dim":1, 
                     "optimizer_fn":torch.optim.Adam,
                     "seed": seed
                    }
    
    input_columns = [] 
    skips = ['low_delta', 'years', 'datetime', 'sent_2']
    for col in train_df.columns:
        if col in skips: continue
        input_columns.append(col) 

    X_train = train_df[input_columns].values 
    Y_train = train_df['high_delta'].values 
    Y_train = Y_train.reshape((-1,1))

    X_val = val_df[input_columns].values
    Y_val = val_df['high_delta'].values 
    Y_val = Y_val.reshape((-1,1))

    X_test = test_df[input_columns].values 
    Y_test = test_df['high_delta'].values  
    Y_test = Y_test.reshape((-1,1))
    
    return X_train, tabnet_params
    
def getRegressor(tabnet_params):
    regressor = TabNetRegressor(**tabnet_params)
    return regressor

In [5]:
def getImportance(df_dir, model_dir):
    seed = model_dir[-4]
    X_train, params = getParams(df_dir, seed)
    regressor = TabNetRegressor()
    regressor.load_model(model_dir)
    
    print(X_train)
    
    #feat_importances = regressor.feature_importances_
    feat_importances = regressor._compute_feature_importances(X_train)
    indices = np.argsort(feat_importances)
    
    plt.figure()
    plt.title("Feature importances")
    plt.barh(range(len(feat_importances)), feat_importances[indices],
           color="r", align="center")
    # If you want to define your own labels,
    # change indices to a list of labels on the following line.
    plt.yticks(range(len(feat_importances)), [FEATS[idx] for idx in indices])
    plt.ylim([-1, len(feat_importances)])
    plt.show()

In [6]:
for ckpt in ckpts:
    print(ckpt)
    getImportance(df_dir, ckpt)

ckpts/no_neutral_8hours/bert-high-9.zip
Device used : cuda
Device used : cuda
[[ 2.         14.         16.         ...  1.04162152  1.34896588
   1.03379448]
 [ 2.         14.         20.         ...  1.05858519  1.02005855
   1.03362294]
 [ 2.         15.          0.         ...  1.03285074  0.60736389
   1.06300886]
 ...
 [ 6.         15.         12.         ...  0.99158888  1.04669938
   0.9944384 ]
 [ 6.         15.         16.         ...  0.98989911  2.52337654
   0.99634937]
 [ 6.         15.         20.         ...  1.00763406  0.76513055
   1.00004009]]


ValueError: too many values to unpack (expected 2)